# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 


# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Completing the Apache Cassandra coding portion of the project. 

## We have the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace 

session.execute("""
CREATE KEYSPACE IF NOT EXISTS udacity 
WITH REPLICATION = 
{ 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)


#### Set Keyspace

In [7]:
#Set KEYSPACE to the keyspace specified above
session.set_keyspace('udacity')

## Creating queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




# Number 1
## Description:
Here the Primary Key has two fields: sessionId is the partition key, and itemInSession is clustering key. 
Partitioning is done by sessionId and within that partition, rows are ordered by the itemInSession.

In [9]:

query = "CREATE TABLE IF NOT EXISTS song_by_user_session "
query = query + "(sessionId int, itemInSession int, artist text,  song_title text, song_length float,  \
                  PRIMARY KEY (sessionId, itemInSession))"

session.execute(query)

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_by_user_session (sessionId, itemInSession, artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [11]:
query = "SELECT ARTIST, SONG_TITLE, SONG_LENGTH from SONG_BY_USER_SESSION WHERE ITEMINSESSION = 4 AND SESSIONID = 338"
rows = session.execute(query)
print('Artist      Song_title                    length')
for row in rows:
    print(row.artist, row.song_title, row.song_length)

Artist      Song_title                    length
Faithless Music Matters (Mark Knight Dub) 495.30731201171875


# Number 2
## Description:
Here the Primary Key has three fields: userId and sessionId are the composite partition keys, and itemInSession is clustering key. 
Partitioning is done by userId and sessionId and within that partition, rows are ordered by the itemInSession.

In [12]:
query2 = "CREATE TABLE IF NOT EXISTS song_by_userid_sessionid "
query2 = query2 + "(userId int, sessionId int, itemInSession int, artist text, song_title text, \
                    user_firstName text, user_lastName text,\
                   PRIMARY KEY ((userId, sessionId), itemInSession))"

session.execute(query2)

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query2 = "INSERT INTO song_by_userid_sessionid (userId, sessionId, itemInSession, artist, song_title,\
                  user_firstName, user_lastName)"
        query2 = query2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query2, (int(line[10]),int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [14]:
query = "SELECT ARTIST, SONG_TITLE,USER_FIRSTNAME, USER_LASTNAME from SONG_BY_USERID_SESSIONID WHERE USERID = 10 \
         AND SESSIONID = 182"
rows = session.execute(query)
print('Artist Song_title First_name Last_name')
for row in rows:
    print(row.artist,row.song_title, row.user_firstname, row.user_lastname)

Artist Song_title First_name Last_name
Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


# Number 3
## Description
Here the Primary Key has two fields: song_title is the partition key, anduserId is clustering key. 
Partitioning is done by song_title and within that partition, rows are ordered by the userId.

In [15]:
query3 = "CREATE TABLE IF NOT EXISTS user_all_hands_against_his_own "
query3 = query3 + "(song_title text, userId int, user_firstName text, user_lastName text, \
                  PRIMARY KEY (song_title, userId))"

session.execute(query3)        

In [16]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query3 = "INSERT INTO user_all_hands_against_his_own (song_title, userId, user_firstName, user_lastName)"
        query3 = query3 + " VALUES (%s, %s, %s, %s)"
        session.execute(query3, (line[9], int(line[10]), line[1], line[4]))

In [17]:
query3 = "SELECT USER_FIRSTNAME, USER_LASTNAME from USER_ALL_HANDS_AGAINST_HIS_OWN WHERE SONG_TITLE = 'All Hands Against His Own'"
rows = session.execute(query3)
print('First name Last name')
for row in rows:
    print(row.user_firstname, row.user_lastname)

First name Last name
Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [18]:
query = "drop table song_by_user_session"
session.execute(query)
query = "drop table song_by_userid_sessionid"
session.execute(query)
query = "drop table user_all_hands_against_his_own"
session.execute(query)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()